<center><h1>Assignment 1</h1></center>

<center><h3>Text generation using N-gram</h2><center>

<center>Saleh Mohammed Al saeed</center>

<center>441014299</center>

In [1]:
# import libraries

import os
import random

import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.util import ngrams
from nltk.probability import FreqDist
from nltk.corpus import stopwords

In [2]:
# Load A corpus into string variable because the data are huge it will take time to predict
corpus_path = os.getcwd() + '/Corpus/A'
corpus = PlaintextCorpusReader(corpus_path, '.*\.txt')
words = corpus.words()
words

['بسم', 'الله', 'الرحمن', 'الرحيم', 'الحمد', 'لله', ...]

In [3]:
# Preprocess text
# stop_words = set(stopwords.words('arabic')) 
# Not needed because result will be not accurate

words_cleaned = [word for word in words if not word in ['*', '_', '"', '.', '،', ',','-','«', '(', ')', '{', '}', '[', ']', ':', '?', '؟'] and not word.isnumeric()]


print(f"Removed words: {(len(words) - len(words_cleaned)):,}")
print(f"Total of cleaned words: {len(words_cleaned):,}")

Removed words: 5,999,258
Total of cleaned words: 22,734,267


In [4]:
# Ask the user to define value n
n = int(input("Enter the value of n: "))

while(n > 6 or n < 2):
    print("n must be between 2 and 6")
    n = int(input("Enter the value of n: "))

Enter the value of n:  6


In [5]:
# Generate N-grams
ngrams_list = ngrams(words_cleaned, n)

In [6]:
# Calculate frequency of N-grams
ngrams_freq = FreqDist(ngrams_list)

In [7]:
# sort the frequent sentence with descending order to become faster to find
ngrams_freq_sorted = sorted(ngrams_freq.items(), key= lambda x: x[1], reverse=True)

In [8]:
# trigram check is for matching the sentence with the trigram and find the best match
def trigram_check(start_slice: int, n_gram: int, trigram: tuple, sentence: str):
    
    context = sentence.split()
    if len(context) >= n_gram:
        context = context[-n_gram:]
        
    if list(trigram[-n_gram:]) == context[-n_gram:]:
        return [False, random.choice(words_cleaned)]
    
    if(context[0] in trigram and trigram.index(context[0]) == start_slice and context[0] != trigram[len(trigram) - 1]):
        for i in range(start_slice, start_slice + n_gram):
            if(i < len(context) and context[i] != trigram[i]):
                return [False, random.choice(words_cleaned)]
        
        try:
            return [True, trigram[trigram.index(context[len(context) - 1]) + 1]]
        except:
            return [False, random.choice(words_cleaned)]
    return [False, random.choice(words_cleaned)]

In [9]:
# predict next word based on the words in sentence
def predict_nextword(sentence: str):
    next_word = None
    
    for trigram, freq in ngrams_freq_sorted:
        for i in range(n):
            next_word = trigram_check(i, n, trigram, sentence)
            if next_word[0]:
                return next_word[1]
            
    return next_word[1]

# Try the function
predict_nextword("الحمد لله رب")

'العالمين'

In [10]:
# Define function to generate output sentence
def generate_sentence(seed_word: str, num_of_words: int):
    sentence = "" + seed_word.strip() + " "
    for i in range(num_of_words - len(seed_word.split())):
        sentence += predict_nextword(sentence) + " "
        
    return sentence

In [11]:
# Ask the user to enter first word to generate sentence
seed_word = input("Enter a word: ")
num_of_words = int(input("Enter the number of word want to generate: "))

Enter a word:  جعلناكم
Enter the number of word want to generate:  6


In [12]:
# Question1: Test the function
generate_sentence(seed_word, num_of_words)

'جعلناكم أمة وسطا لتكونوا شهداء على '

In [13]:
# Question2: the 10 high frequent (trigram) in the corpus

# reload the whole corpus
corpus_path = os.getcwd() + '/Corpus/'
corpus = PlaintextCorpusReader(corpus_path, '.*\.txt')
words = corpus.words()

In [14]:
# Frequent n-gram and the most common n-grams
ngrams_list = ngrams(words, n)
ngrams_freq = FreqDist(ngrams_list)
most_common_trigram = ngrams_freq.most_common(10)

In [15]:
# Printing the n-grams before removing the symbols
for trigram, count in most_common_trigram:
    print(f"{trigram}: {count}")

('رسول', 'الله', 'صلى', 'الله', 'عليه', 'وسلم'): 83177
('.', '.', '.', '.', '.', '.'): 33407
('الله', 'صلى', 'الله', 'عليه', 'وسلم', ':'): 16009
('-', 'صلى', 'الله', 'عليه', 'وسلم', '-'): 15115
('عن', 'النبي', 'صلى', 'الله', 'عليه', 'وسلم'): 13075
('صلى', 'الله', 'عليه', 'وسلم', ':', '«'): 12112
('قال', 'رسول', 'الله', 'صلى', 'الله', 'عليه'): 11949
('الله', 'صلى', 'الله', 'عليه', 'وسلم', '،'): 9120
(':', 'قال', 'رسول', 'الله', 'صلى', 'الله'): 8998
('__________', '[', 'تعليق', 'مصطفى', 'البغا', ']'): 8115


In [16]:
# clean the trigram
symbols = ['-[', '__________', '*', '/','_', '"', '.', '،', ',','-','«', '(', ')', '{', '}', '[', ']', ':', '?', '؟']
words_cleaned = [word for word in words if not word in symbols and not word.isnumeric()]

# n-grams after removing the symbols
ngrams_list = ngrams(words_cleaned, n)
ngrams_freq = FreqDist(ngrams_list)
most_common_trigram = ngrams_freq.most_common(10)

In [17]:
# Printing the n-grams After removing the symbols
for trigram, count in most_common_trigram:
    print(f"{trigram}: {count}")

('رسول', 'الله', 'صلى', 'الله', 'عليه', 'وسلم'): 91285
('عن', 'النبي', 'صلى', 'الله', 'عليه', 'وسلم'): 14549
('قال', 'رسول', 'الله', 'صلى', 'الله', 'عليه'): 12701
('أن', 'رسول', 'الله', 'صلى', 'الله', 'عليه'): 9062
('قال', 'قال', 'رسول', 'الله', 'صلى', 'الله'): 8847
('النبي', 'صلى', 'الله', 'عليه', 'وسلم', 'قال'): 7631
('أن', 'النبي', 'صلى', 'الله', 'عليه', 'وسلم'): 7178
('الله', 'صلى', 'الله', 'عليه', 'وسلم', 'قال'): 6873
('عن', 'ابن', 'أبي', 'نجيح', 'عن', 'مجاهد'): 6348
('عن', 'رسول', 'الله', 'صلى', 'الله', 'عليه'): 4890


In [18]:
# print the most common to a text file
with open('most_common_trigram.txt', 'w', encoding='utf-8') as f:
    for trigram, freq in most_common_trigram:
        f.write(' '.join(trigram) + '\t' + str(freq) + '\n')